In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
W = 848
H = 480

# Configure depth and color streams
sn='031222070617'

pipeline = rs.pipeline()
config = rs.config()
config.enable_device(sn)
#config.enable_stream(rs.stream.depth, W, H, rs.format.z16, 30)
#config.enable_stream(rs.stream.color, W, H, rs.format.bgr8, 30)

print("[INFO] start streaming...")
pipeline.start(config)

aligned_stream = rs.align(rs.stream.color) # alignment between color and depth
point_cloud = rs.pointcloud()


[INFO] start streaming...


In [4]:
import argparse
import cv2

from yolo import YOLO


network="normal"
hands=2
device = 0
if network == "normal":
    print("loading yolo...")
    yolo = YOLO("models/cross-hands.cfg", "models/cross-hands.weights", ["hand"])
elif network == "prn":
    print("loading yolo-tiny-prn...")
    yolo = YOLO("models/cross-hands-tiny-prn.cfg", "models/cross-hands-tiny-prn.weights", ["hand"])
elif network == "v4-tiny":
    print("loading yolov4-tiny-prn...")
    yolo = YOLO("models/cross-hands-yolov4-tiny.cfg", "models/cross-hands-yolov4-tiny.weights", ["hand"])
else:
    print("loading yolo-tiny...")
    yolo = YOLO("models/cross-hands-tiny.cfg", "models/cross-hands-tiny.weights", ["hand"])

yolo.size = int(416)
yolo.confidence = float(0.2)

print("starting webcam...")
cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

if vc.isOpened():  # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    width, height, inference_time, results = yolo.inference(frame)

    # display fps
    cv2.putText(frame, f'{round(1/inference_time,2)} FPS', (15,15), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,255,255), 2)

    # sort by confidence
    results.sort(key=lambda x: x[2])

    # how many hands should be shown
    hand_count = len(results)
    if hands != -1:
        hand_count = int(hands)

    # display hands
    for detection in results[:hand_count]:
        id, name, confidence, x, y, w, h = detection
        cx = x + (w / 2)
        cy = y + (h / 2)

        # draw a bounding box rectangle and label on the image
        color = (0, 255, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        xmin = x
        ymin = y
        xmax = x+w
        ymax = y+h
        
        text = "%s (%s)" % (name, round(confidence, 2))
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)

    cv2.imshow("preview", frame)

    rval, frame = vc.read()

    key = cv2.waitKey(20)
    if key == 27:  # exit on ESC
        break

cv2.destroyWindow("preview")
vc.release()

loading yolo...
starting webcam...


In [44]:

from yolo import YOLO
yolo = YOLO("models/cross-hands-tiny.cfg", "models/cross-hands-tiny.weights", ["hand"])
yolo.size = int(256)
yolo.confidence = float(0.2)
def getHandDetection(frame,depth_frame,hands):
    
    width, height, inference_time, results = yolo.inference(frame)

    # display fps
    cv2.putText(frame, f'{round(1/inference_time,2)} FPS', (15,15), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,255,255), 2)

    # sort by confidence
    results.sort(key=lambda x: x[2])

    # how many hands should be shown
    hand_count = len(results)
    if hands != -1:
        hand_count = int(hands)

    # display hands
    for detection in results[:hand_count]:
        id, name, confidence, x, y, w, h = detection
        cx = x + (w / 2)
        cy = y + (h / 2)

        # draw a bounding box rectangle and label on the image
        color = (0, 255, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        xmin_depth = x
        ymin_depth = y
        xmax_depth = x+w
        ymax_depth = y+h
        depth = np.asanyarray(depth_frame.get_data())
        # Crop depth data:
        depth = depth[xmin_depth:xmax_depth,ymin_depth:ymax_depth].astype(float)
        showBoxonDepth(depth_frame, xmin_depth,xmax_depth,ymin_depth,ymax_depth)

        # Get data scale from the device and convert to meters
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        depth = depth * depth_scale
        dist,_,_,_ = cv2.mean(depth)
        #print("Detected {0:.3} meters away.".format(dist))
        distanceString = "Detected {0:.3} meters away.".format(dist)
        text = "%s (%s)" % (name, round(confidence, 2))
        cv2.putText(frame, text+distanceString, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
    
    cv2.imshow("preview", frame)
    
    return frame 

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

font = cv2.FONT_HERSHEY_SIMPLEX
 
# fontScale
fontScale = 1
   
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
def getFingers(frame):
    with mp_hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.5) as hands:
        

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        #image = cv2.flip(image, 1)

        # Set flag
        image.flags.writeable = False

        # Detections
        results = hands.process(image)

        # Set flag to true
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detections
        #print(results)
        image_height, image_width, _ = image.shape


        # Rendering results
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                xIndexFinger =hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width
                yIndexFinger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y* image_height
                xMiddleFinger =hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width
                yMiddleFinger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y* image_height
                cv2.putText(image, 'o', (int(xIndexFinger)-10,int(yIndexFinger)-10), font, fontScale, (255, 0, 0), 4, cv2.LINE_AA)
                #cv2.putText(image, 'o', (int(xMiddleFinger)-10,int(yMiddleFinger)-10), font, fontScale, (255, 255, 0), 4, cv2.LINE_AA)
                return image, xIndexFinger, yIndexFinger
        return image, False, False
        
        
        

In [4]:
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import mediapipe as mp

W = 848
H = 480

from yolo import YOLO
#yolo = YOLO("models/cross-hands.cfg", "models/cross-hands.weights", ["hand"])
yolo = YOLO("models/cross-hands-tiny.cfg", "models/cross-hands-tiny.weights", ["hand"])
yolo.size = int(416)
yolo.confidence = float(0.2)


sn='031222070617'

pipeline = rs.pipeline()
config = rs.config()
config.enable_device(sn)

#config.enable_stream(rs.stream.depth, W, H, rs.format.z16, 30)
#config.enable_stream(rs.stream.color, W, H, rs.format.bgr8, 30)

dec_filter = rs.decimation_filter()  # Decimation-reduce df density
spat_filter = rs.spatial_filter()    # Spatial-spatial smoothing
temp_filter = rs.temporal_filter()   # Temporal-reduces temporal noise

print("[INFO] start streaming...")
profile = pipeline.start(config)
#device = profile.get_device()
#depth_sensor = device.first_depth_sensor()
#device.hardware_reset()

depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
align = rs.align(rs.stream.color)
point_cloud = rs.pointcloud()
for x in range(10):
    pipeline.wait_for_frames()


frames = pipeline.wait_for_frames()
#frames = aligned_stream.process(frames)
depth_frame = frames.get_depth_frame()
color_frame = frames.get_color_frame()
#points = point_cloud.calculate(depth_frame)
#color_frame = frameset_before.get_color_frame()
frames = dec_filter.process(frames).as_frameset()
frames = spat_filter.process(frames).as_frameset()
frames = temp_filter.process(frames).as_frameset()


while True:
    frames = pipeline.wait_for_frames()
   
    
    
    aligned_frames = align.process(frames)       # Get aligned frames
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = frames.get_color_frame()
        
    

    
    #verts = np.asanyarray(points.get_vertices()).view(np.float32).reshape(-1, W, 3)  # xyz
    # Convert images to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    
    frame = color_image
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hands = 1
    
    colorizer = rs.colorizer()
    colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())
    depth = np.asanyarray(depth_frame.get_data())
    # Get data scale from the device and convert to meters
    
    
    width, height, inference_time, results = yolo.inference(frame)
    frame,indexX,indexY = getFingers(frame)


    # display fps
    cv2.putText(frame, f'{round(1/inference_time,2)} FPS', (15,15), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,255,255), 2)

    # sort by confidence
    results.sort(key=lambda x: x[2])

    # how many hands should be shown
    hand_count = len(results)
    if hands != -1:
        hand_count = int(hands)
   

    # display hands
    for detection in results[:hand_count]:
        id, name, confidence, x, y, w, h = detection
        cx = x + (w / 2)
        cy = y + (h / 2)

        # draw a bounding box rectangle and label on the image
        color = (0, 255, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        xmin_depth = x+round(w/4)
        ymin_depth = y+round(h/4)
        xmax_depth = x+round(w*.75)
        ymax_depth = y+round(h*.75)      
        
        
        
        #depth = np.asanyarray(depth_frame.get_data())
        # Crop depth data:
        depth_hand = depth[ymin_depth:ymax_depth,xmin_depth:xmax_depth].astype(float)
        depth_hand_scaled = depth_hand * depth_scale
        dist,_,_,_ = cv2.mean(depth_hand_scaled)
        #print("Detected {0:.3} meters away.".format(dist))
        distanceString = "Detected {0:.3} meters away.".format(dist)
        
        
        cv2.rectangle(colorized_depth, (xmin_depth, ymin_depth), 
                     (xmax_depth, ymax_depth), (255, 255, 255), 2)
        
        cv2.putText(colorized_depth, distanceString, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
        cv2.imshow("Depth Frame",colorized_depth)
        
        text = "%s (%s)" % (name, round(confidence, 2))
        cv2.putText(frame, text+distanceString, (15, 65), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
        images = np.hstack((frame, colorized_depth))
        plt.imshow(images)
    
    if indexX:
        indexDist=(depth[int(indexY),int(indexX)].astype(float))*depth_scale
        iDistanceSTR = "Detected {0:.3} meters away.".format(indexDist)
        cv2.putText(frame,"Index Finger "+ iDistanceSTR,(15,35),cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 0, 0), 2 )

    
    
        
    cv2.imshow("preview", frame)  
    
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
#image_expanded = np.expand_dims(color_image, axis=0)
pipeline.stop()

[INFO] start streaming...


RuntimeError: Frame didn't arrive within 5000

In [69]:
depth_hand = depth[ymin_depth:ymax_depth,xmin_depth:xmax_depth].astype(float)
depth_hand_scaled = depth_hand * depth_scale
dist,_,_,_ = cv2.mean(depth_hand_scaled)
print(dist)

0.947356038591627


In [5]:
#pipe.stop()
pipeline.stop()

In [2]:
# Update color and depth frames:
#aligned_depth_frame = frameset.get_depth_frame()

def showImages(color_frame, aligned_depth_frame):
    colorizer = rs.colorizer()

    colorized_depth = np.asanyarray(colorizer.colorize(aligned_depth_frame).get_data())
    color = np.asanyarray(color_frame.get_data())
    # Show the two frames together:
    images = np.hstack((color, colorized_depth))
    plt.imshow(images)


In [20]:
def showBoxonDepth( depth_frame, xmin_depth,xmax_depth,ymin_depth,ymax_depth):
    colorizer = rs.colorizer()
    colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())
    plt.imshow(colorized_depth)
    cv2.rectangle(colorized_depth, (xmin_depth, ymin_depth), 
                 (xmax_depth, ymax_depth), (255, 255, 255), 2)
    plt.imshow(colorized_depth)

In [3]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        #print(results)
        image_height, image_width, _ = image.shape
        
        
        # Rendering results
        if results.multi_hand_landmarks:
           # print(results.multi_hand_landmarks)
           
            for hand_landmarks in results.multi_hand_landmarks:
                xIndexFinger =hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width
                yIndexFinger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y* image_height
                xMiddleFinger =hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width
                yMiddleFinger = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y* image_height
                cv2.putText(image, 'o', (int(xIndexFinger)-10,int(yIndexFinger)-10), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'o', (int(xMiddleFinger)-10,int(yMiddleFinger)-10), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 0), 4, cv2.LINE_AA)
               
               
            
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()